In [1]:
import pandas as pd

In [2]:
jan_25 = pd.read_csv("../original_data/traffic_density_202501.csv")

In [3]:
jan_25.head()

,DATE_TIME,LATITUDE,LONGITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
0,2025-01-01 00:00:00,41.157532,28.888550,sxkd1k,58,107,79,11
1,2025-01-01 00:00:00,41.003723,29.306030,sxkc6b,62,104,81,19
2,2025-01-01 00:00:00,41.009216,29.295044,sxkc69,77,103,88,7
3,2025-01-01 00:00:00,41.003723,29.295044,sxkc68,64,110,85,19
4,2025-01-01 00:00:00,41.020203,29.273071,sxkc65,56,127,90,73


In [4]:
jan_25 = jan_25.drop(columns=["GEOHASH", "MINIMUM_SPEED", "MAXIMUM_SPEED"])

In [5]:
jan_25.head()
print(jan_25.describe())
jan_25.info()

           LATITUDE     LONGITUDE  AVERAGE_SPEED  NUMBER_OF_VEHICLES
count  1.763963e+06  1.763963e+06   1.763963e+06        1.763963e+06
mean   4.106263e+01  2.888683e+01   5.617976e+01        8.896445e+01
std    1.008242e-01  3.561361e-01   2.564569e+01        1.120803e+02
min    4.076752e+01  2.796570e+01   2.000000e+00        1.000000e+00
25%    4.099823e+01  2.869080e+01   3.300000e+01        1.700000e+01
50%    4.105865e+01  2.893250e+01   5.400000e+01        4.900000e+01
75%    4.113007e+01  2.914124e+01   7.900000e+01        1.150000e+02
max    4.134430e+01  2.963562e+01   1.920000e+02        1.480000e+03
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1763963 entries, 0 to 1763962
Data columns (total 5 columns):
 #   Column              Dtype  
---  ------              -----  
 0   DATE_TIME           object 
 1   LATITUDE            float64
 2   LONGITUDE           float64
 3   AVERAGE_SPEED       int64  
 4   NUMBER_OF_VEHICLES  int64  
dtypes: float64(2), int64(2), object

In [6]:
jan_25['DATE'] = pd.to_datetime(jan_25['DATE_TIME']).dt.date
jan_25.drop(columns = ['DATE_TIME'], inplace = True)

daily_data = jan_25.groupby(['DATE', 'LATITUDE', 'LONGITUDE']).agg({
    'AVERAGE_SPEED': 'mean',
    'NUMBER_OF_VEHICLES': 'sum'
}).reset_index()
print(daily_data.head())
print(jan_25.describe())
jan_25.info()

         DATE   LATITUDE  LONGITUDE  AVERAGE_SPEED  NUMBER_OF_VEHICLES
0  2025-01-01  40.767517  29.426880      35.291667                 474
1  2025-01-01  40.773010  29.415894      63.875000                 210
2  2025-01-01  40.773010  29.426880      53.291667                 541
3  2025-01-01  40.778503  29.415894      63.791667                 586
4  2025-01-01  40.783997  29.415894      55.291667                2106
           LATITUDE     LONGITUDE  AVERAGE_SPEED  NUMBER_OF_VEHICLES
count  1.763963e+06  1.763963e+06   1.763963e+06        1.763963e+06
mean   4.106263e+01  2.888683e+01   5.617976e+01        8.896445e+01
std    1.008242e-01  3.561361e-01   2.564569e+01        1.120803e+02
min    4.076752e+01  2.796570e+01   2.000000e+00        1.000000e+00
25%    4.099823e+01  2.869080e+01   3.300000e+01        1.700000e+01
50%    4.105865e+01  2.893250e+01   5.400000e+01        4.900000e+01
75%    4.113007e+01  2.914124e+01   7.900000e+01        1.150000e+02
max    4.134430e+01  2

In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

districts = gpd.read_file("../data/istanbul-districts.json") 

points = gpd.GeoDataFrame(
    jan_25,
    geometry=gpd.points_from_xy(jan_25["LONGITUDE"], jan_25["LATITUDE"]),
    crs="EPSG:4326"
)

points_with_district = gpd.sjoin(
    points,
    districts[["name", "geometry"]].rename(columns={"name": "DISTRICT"}),
    how="left",
    predicate="within"
)

jan_25["DISTRICT"] = points_with_district["DISTRICT"]

In [8]:
jan_25.head()

,LATITUDE,LONGITUDE,AVERAGE_SPEED,NUMBER_OF_VEHICLES,DATE,DISTRICT
0,41.157532,28.888550,79,11,2025-01-01,Eyüpsultan
1,41.003723,29.306030,81,19,2025-01-01,Pendik
2,41.009216,29.295044,88,7,2025-01-01,Pendik
3,41.003723,29.295044,85,19,2025-01-01,Pendik
4,41.020203,29.273071,90,73,2025-01-01,Sancaktepe


In [9]:
code_dict = {
    "Adalar": 0,
    "Arnavutköy": 1,
    "Ataşehir": 2,
    "Avcılar": 3,
    "Bağcılar": 4,
    "Bahçelievler": 5,
    "Bakırköy": 6,
    "Başakşehir": 7,
    "Bayrampaşa": 8,
    "Beşiktaş": 9,
    "Beykoz": 10,
    "Beylikdüzü": 11,
    "Beyoğlu": 12,
    "Çatalca": 13,
    "Çekmeköy": 14,
    "Esenler": 15,
    "Esenyurt": 16,
    "Eyüpsultan": 17,
    "Fatih": 18,
    "Gaziosmanpaşa": 19,
    "Güngören": 20,
    "Kadıköy": 21,
    "Kağıthane": 22,
    "Kartal": 23,
    "Küçükçekmece": 24,
    "Maltepe": 25,
    "Pendik": 26,
    "Sancaktepe": 27,
    "Sarıyer": 28,
    "Şile": 29,
    "Şişli": 30,
    "Sultanbeyli": 31,
    "Sultangazi": 32,
    "Tuzla": 33,
    "Ümraniye": 34,
    "Üsküdar": 35,
    "Zeytinburnu": 36,
    "Silivri": 37,
    "Çekmeköy": 38
}

In [10]:
jan_25["DISTRICT"] = jan_25["DISTRICT"].apply(lambda x: code_dict[x] if x in code_dict else None).astype("Int64")

jan_25.dropna(subset=["DISTRICT"], inplace=True)

In [11]:
jan_25.head()
jan_25.drop(columns=["LATITUDE", "LONGITUDE"], inplace=True)

In [12]:
jan_25.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1585723 entries, 0 to 1763962
Data columns (total 4 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   AVERAGE_SPEED       1585723 non-null  int64 
 1   NUMBER_OF_VEHICLES  1585723 non-null  int64 
 2   DATE                1585723 non-null  object
 3   DISTRICT            1585723 non-null  Int64 
dtypes: Int64(1), int64(2), object(1)
memory usage: 62.0+ MB


In [13]:
jan_25.head()

,AVERAGE_SPEED,NUMBER_OF_VEHICLES,DATE,DISTRICT
0,79,11,2025-01-01,17
1,81,19,2025-01-01,26
2,88,7,2025-01-01,26
3,85,19,2025-01-01,26
4,90,73,2025-01-01,27


In [14]:
print(jan_25[['NUMBER_OF_VEHICLES', 'AVERAGE_SPEED']].describe())

       NUMBER_OF_VEHICLES  AVERAGE_SPEED
count        1.585723e+06   1.585723e+06
mean         9.115278e+01   5.525047e+01
std          1.145804e+02   2.589347e+01
min          1.000000e+00   2.000000e+00
25%          1.800000e+01   3.200000e+01
50%          5.000000e+01   5.200000e+01
75%          1.180000e+02   7.900000e+01
max          1.480000e+03   1.920000e+02


In [15]:
q_low,  q_hi   = 0.05, 0.95
nv_lo, nv_hi   = jan_25["NUMBER_OF_VEHICLES"].quantile([q_low, q_hi])
sp_lo, sp_hi   = jan_25["AVERAGE_SPEED"].quantile([q_low, q_hi])

jan_25["v_norm"]   = ((jan_25["NUMBER_OF_VEHICLES"] - nv_lo) / (nv_hi - nv_lo)).clip(0, 1)
jan_25["s_norm"]   = ((jan_25["AVERAGE_SPEED"]     - sp_lo) / (sp_hi - sp_lo)).clip(0, 1)

In [16]:
jan_25["s_inv"] = 1 - jan_25["s_norm"]

In [17]:
jan_25["congestion"] = 0.5*jan_25["v_norm"] + 0.5*jan_25["s_inv"]

In [18]:
jan_25["congestion"] = jan_25["congestion"].clip(0, 1)

In [19]:
jan_25.head()
jan_25.drop(columns=["NUMBER_OF_VEHICLES", "AVERAGE_SPEED", "v_norm", "s_norm", "s_inv"], inplace=True)

In [20]:
jan_25.head()

,DATE,DISTRICT,congestion
0,2025-01-01,17,0.124264
1,2025-01-01,26,0.123529
2,2025-01-01,26,0.058843
3,2025-01-01,26,0.097213
4,2025-01-01,27,0.148169
